# MDA 2021
## Pyspark Sample Code
-----------------------------------------------------------------

## Setup
--------------------------------------------------

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to processing data

**Make sure to follow the interactive instructions.**

In [2]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


## Check and extract data
--------------------------------------------------

In [ ]:
!ls '/content/drive/My Drive/Colab Notebooks/MDA/HW3/'

 checkstatus.csv   MDA_2021.ipynb       SystemID.csv
 CompanyID.csv	   Sample_Data.zip      Traffic.csv
 Data.zip	   Sample_Traffic.csv  'پروژه MDA2021.pdf'


In [2]:
# !unzip "/content/drive/My Drive/Colab Notebooks/MDA/HW3/Sample_Data.zip" -d "/content/drive/My Drive/Colab Notebooks/MDA/HW3"
!unzip "./Sample_Data.zip" -d "./"

Archive:  ./Sample_Data.zip
  inflating: ./Sample_Traffic.csv    


the cells above, extract data which is in '/content/drive/My Drive/Test' to /content/drive/My Drive/Test/Traffic.csv  

## Initializing Spark and read data
--------------------------------------------------

In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
spark = SparkSession \
    .builder \
    .appName("Spark_Processor") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

schema = StructType([ \
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

21/12/05 20:48:27 WARN Utils: Your hostname, amin-X556UQK resolves to a loopback address: 127.0.1.1; using 192.168.1.15 instead (on interface wlp3s0)
21/12/05 20:48:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/05 20:48:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h3 dir="rtl">
 در این قسمت، لطفا آدرس فایل 
csv
 را به جای آدرس قرار داده شده جایگذاری کنید.  
  
</h3> 

In [2]:
# df=spark.read.csv('/content/drive/My Drive/Colab Notebooks/MDA/HW3/Sample_Traffic.csv',header=True,schema=schema)
df=spark.read.csv('Sample_Traffic.csv',header=True,schema=schema)
# df.select('DEVICE_CODE').distinct().count()
df.show(5)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     200501|       81|       10477885|     10477885|               5|       161|2021-06-01 03:54:39|
|        155|       81|       87625017|     87625017|               5|       161|2021-06-01 04:14:21|
|     631757|       81|        8652928|      8652928|               5|       161|2021-06-01 03:58:57|
|     631757|       81|        8548123|      8548123|               5|       161|2021-06-01 04:01:38|
|     631757|       81|       24715264|     24715264|               5|       161|2021-06-01 03:56:57|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 5 rows



<h3 dir="rtl">
 با استفاده از یک بار فرایند 
 map
 ، هر ردیف از 
 df
 را به صورت کلید پلاک خودرو، و زمان عبور از مسیر در می‌آوریم. مقدار این کلید را نیز برابر کد دستگاه (
     که نمایانگر مسیر عبوری است 
 ) 
 می‌گذاریم. 
 rdd
 به دست آمده به ازای هر مسیر در هر روز، یک ردیف خواهد داشت. 
</h3> 


In [3]:
rdd = df.rdd.map(lambda x: ((x['FINAL_CAR_KEY'], x['PASS_DAY_TIME'].date()),  x['DEVICE_CODE']))
rdd.take(5)

[(('10477885', datetime.date(2021, 6, 1)), 200501),
 (('87625017', datetime.date(2021, 6, 1)), 155),
 (('8652928', datetime.date(2021, 6, 1)), 631757),
 (('8548123', datetime.date(2021, 6, 1)), 631757),
 (('24715264', datetime.date(2021, 6, 1)), 631757)]

<h3 dir="rtl">
 اگر 
 rdd
 بالا را به کمک کلیدش 
 group_by
 کنیم، برای هر ماشین در هر روز خاص، تمام مسیر‌هایی که عبور کرده در یک لیست خواهیم داشت. البته به دلیل ماهیت مسئله 
 frequent_itemsets
 ، 
 به جای لیست از
 set
 استفاده کرده‌ام، تا مسیر‌های تکراری نداشته باشیم. هرچند می‌توان مسئله را به گونه‌ای تعریف کرد که ترتیب و یا تعداد تکرار مسیرها تاثیر گذار باشد، اما با توجه به خروجی‌ای که ما می‌خوایم، این فرض چیزی از کلیت مسئله نمی‌کاهد.  
</h3> 


In [4]:
rdd_group = rdd.groupByKey().mapValues(set)
rdd_group.take(5)

[(('69939810', datetime.date(2021, 6, 1)), {206602, 631830, 900233}),
 (('11046172', datetime.date(2021, 6, 1)), {206602}),
 (('29077699', datetime.date(2021, 6, 1)),
  {119,
   128,
   206602,
   208602,
   210602,
   631367,
   631763,
   631829,
   900135,
   900233,
   900234,
   900240,
   900246,
   900266,
   22010111,
   22010123}),
 (('40682798', datetime.date(2021, 6, 1)), {206602}),
 (('48823778', datetime.date(2021, 6, 1)), {206602})]

# A-Priori

## Step by Step

<h3 dir="rtl">
 برای داشتن سبدها، نیازی به نگه داشتن پلاک و روز نداریم. بنابرین آن‌ها را نگه نمی‌داریم و سبدها را به دست می‌آوریم.  
</h3> 


In [27]:
import numpy as np

baskets_rdd = rdd_group.map(lambda x: x[1])
baskets_rdd.take(2)

[{206602, 631830, 900233}, {206602}]

<h3 dir="rtl">
 این قسمت از کد، برای تست کردن استفاده می‌شود، که به کمک آن می‌توانیم یک نمونه از داده مسئله بگیریم تا اجرای کد سریع تر باشد. متغیر
 THRESHOLD
 نیز نسبتی از کل سبدهاست که از آن بیشتر مجموعه را پر تکرار تشخیص می‌دهیم. در مورد نتایج و پارامترهای استفاده شده در انتهای کد توضیح خواهم داد. 
</h3> 


In [30]:
SAMPLE = True
SAMPLE_SIZE = 0.01
THRESHOLD = 0.001

if SAMPLE:
  baskets_rdd = baskets_rdd.sample(True, SAMPLE_SIZE)
  baskets_rdd = baskets_rdd.coalesce(10)
  baskets_rdd.cache()


<h3 dir="rtl">
 در دو بلاک بعدی، ابتدا تعداد کل سبدها را می‌شماریم، و سپس با استفاده از 
 THRESHOLD
 ، حداقل ساپورت را به دست می‌آوریم. مقدار 
 MIN_COUNT
 را نیز در انتهای گذارش ذکر خواهم کرد.  
</h3> 


In [31]:
BASKETS_COUNT = baskets_rdd.count()
BASKETS_COUNT

165

In [32]:
MIN_COUNT = int(THRESHOLD * BASKETS_COUNT)
MIN_COUNT

0

<h3 dir="rtl">
 در این مرحله ابتدا تعداد تکرار هر یک از 
 item
 ها را به دست می‌آورم و سپس با کمک حداقل ساپورت، 
 item
 های پر تکرار را به دست می‌آورم.  
</h3> 


In [33]:
frequent_items_rdd = baskets_rdd.flatMap(lambda basket: [((item,),1) for item in basket]).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] > MIN_COUNT)
frequent_items_rdd.take(5)

[((631363,), 2),
 ((900171,), 3),
 ((119,), 5),
 ((22010087,), 1),
 ((22010095,), 1)]

<h3 dir="rtl">
 این کد تعداد 
 item
 های پرتکرار را محاسبه و چاپ می کند. 
</h3> 


In [34]:
FREQUENT_ITEMS_COUNT = frequent_items_rdd.count()
FREQUENT_ITEMS_COUNT

202

<h3 dir="rtl">
 <ul dir="rtl">
  <li dir="rtl">
  generate_combinations 
    
  </li> 
  این تابع، به عنوان ورودی 
  itemset
  های پرتکرار مرحله‌ی قبلی را می‌گیرد، و کاندید‌های مرحله‌ی بعد را می‌سازد. این کار به به این صورت انجام می‌دهد که ابتدا 
  item
  های پرتکرار باقی‌مانده در کل 
  itemset
  های پرتکرار را به دست می‌آورد و مرتب می‌کند. سپس به ازای هر 
  itemset
  پرتکرار، بزرگترین عنصر آن را می‌گیرد، و هر عضو مجموعه‌ی 
  item
  های باقی مانده که از این عضو بزرگتر است را به انتهای این مجموعه اضافه می‌کند و یک کاندید جدید در نظر می‌گیرد. منطق این تابع به این صورت است که نیازی نیست عناصر کوچک‌تر را به این مجموعه‌ها اضافه کنیم. برای مثال فرض کنید مجموعه‌ی 
  [b,c,d]
  در مرحله‌ی قبلی پرتکرار بوده باشد، و مجموعه‌ی عناصر پرتکرار باقی مانده نیز
  [a,b,c,d,z]
  باشد (که این عناصر به ترتیب مرتب شده‌اند). از عناصر پرتکرار باقی مانده، تنها 
  z
  است که از تمام اعضای این مجموعه بزرگ‌تر است. حال یک کاندید معرفی شده، 
  [b,c,d,z]
  است. اما 
  [a,b,c,d]
  به عنوان کاندید مطرح نمی‌شود. فرض کنید پرتکرار باشد. می‌دانیم تمام زیرمجموعه‌های یک مجموعه‌ی پرتکرار نیز پرتکرارند. حال اگر 
  [a,b,c]
  پرتکرار بوده باشد،‌ پس در لیست پرتکرارها موجود است و ترکیب‌های آن را هنگام بررسی 
  [a,b,c]
  می‌سازیم. اما اگر پرتکرار نباشد، پس 
  [a,b,c,d]
  نیز پرتکرار نیست و به درستی آن را نساخته‌ایم. 
 
<li dir="rtl">
      get_new_frequents
</li> 
 این تابع تمام سبدها را به عنوان ورودی می‌گیرد، و 
 itemset
 های پرتکرار مرحله‌ی جدید را به دست می‌آورد. در این راه از تابع زیر به عنوان تابع
 map
 کمک می‌گیرد: 

<li dir="rtl">
  count_freq
</li> 
   این تابع، یک سبد به عنوان ورودی می‌گیرد، و به ازای هر کدام از کاندید‌های آن مرحله، اگر آن کاندید در آن سبد موجود باشد، یک خروجی به صورت 
   (candidate,1) 
   می‌سازد و خروجی می‌دهد. در نهایت، با شمردن تمام کاندیدها، می‌توانیم 
   itemset
   های پرتکرار را به دست آوریم. ‌
 
 </ul> 
</h3> 


In [35]:
def generate_combinations(old_combinations):
  """
  Input old_frequent_itemsets, and create new candidates from it
  """

  def generate_combinations_util(old_combination):
    """
    lambda function that maps an old combination to a number of new candidate combinations
    """
    old_combination = old_combination[0]
    old_combination_max_item = old_combination[-1]

    # Can do here numpy way
    bigger_items = remaining_items[remaining_items>old_combination_max_item]
    new_candidates = []
    for x in bigger_items:
      new_candidates.append( old_combination + (x,) )

    return new_candidates

  remaining_items = np.array(old_combinations.flatMap(lambda x: x[0]).distinct().sortBy(lambda x: x).collect())
  # print('remaining items: ', remaining_items)

  new_combinations = old_combinations.flatMap(generate_combinations_util)
  return new_combinations

def get_new_frequents(candidates):
  # frequent_itemsets_rdd = frequent_itemsets_rdd.filter(lambda x: set(x[0]) <= set(x[1])).map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>MIN_COUNT)
  
  _candidates = candidates.collect()
  def count_freq(basket):
    candidates_present = []
    for candidate in _candidates:
      if set(candidate) <= set(basket):
        candidates_present.append( (candidate,1) )
    
    return candidates_present

  frequent_itemsets_rdd = baskets_rdd.flatMap(count_freq).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>MIN_COUNT)
  
  return frequent_itemsets_rdd

k = 1
frequent_itemsets_rdd = frequent_items_rdd
while frequent_itemsets_rdd.count() != 0:
  print('-----------------loop start----------------------')
  k += 1
  candidates = generate_combinations(frequent_itemsets_rdd)
  frequent_itemsets_rdd = get_new_frequents(candidates)
  print('Itemsets of size ', k, ', count: ', frequent_itemsets_rdd.count())
  print('sample: ')
  print(frequent_itemsets_rdd.take(10))



-----------------loop start----------------------
Itemsets of size  2 , count:  732
sample: 
[((208602, 22010060), 1), ((631633, 900269), 1), ((631357, 631633), 1), ((631357, 900269), 1), ((119, 900171), 1), ((22010117, 100700845), 1), ((900233, 22010117), 1), ((900233, 100700845), 1), ((900149, 900233), 1), ((900149, 22010117), 1)]
-----------------loop start----------------------


Itemsets of size  3 , count:  1772
sample: 
[((900171, 900233, 22010117), 1), ((900171, 900233, 100700845), 1), ((900171, 22010117, 100700845), 1), ((119, 900149, 900233), 1), ((119, 900149, 22010117), 1), ((119, 900149, 100700845), 1), ((119, 900233, 22010117), 1), ((119, 900233, 100700845), 1), ((119, 22010117, 100700845), 1), ((900124, 900160, 900233), 1)]
-----------------loop start----------------------


Itemsets of size  4 , count:  3518
sample: 
[((119, 900171, 900233, 22010117), 1), ((119, 900171, 900233, 100700845), 1), ((119, 900171, 22010117, 100700845), 1), ((900149, 900233, 22010117, 100700845), 1), ((119, 900124, 900160, 900233), 1), ((119, 900124, 900160, 22010117), 1), ((119, 900124, 900160, 100700845), 1), ((900124, 900149, 900160, 900233), 1), ((900124, 900149, 900160, 22010117), 1), ((900124, 900149, 900160, 100700845), 1)]
-----------------loop start----------------------


Itemsets of size  5 , count:  5823
sample: 
[((119, 900149, 900233, 22010117, 100700845), 1), ((900124, 900160, 900233, 22010117, 100700845), 1), ((900149, 900171, 900233, 22010117, 100700845), 1), ((119, 900124, 900149, 900160, 900233), 1), ((119, 900124, 900149, 900160, 22010117), 1), ((119, 900124, 900149, 900160, 100700845), 1), ((119, 900124, 900160, 900171, 900233), 1), ((119, 900124, 900160, 900171, 22010117), 1), ((119, 900124, 900160, 900171, 100700845), 1), ((900124, 900149, 900160, 900171, 900233), 1)]
-----------------loop start----------------------


Itemsets of size  6 , count:  7960
sample: 
[((119, 900124, 900160, 900233, 22010117, 100700845), 1), ((900124, 900149, 900160, 900233, 22010117, 100700845), 1), ((119, 900149, 900171, 900233, 22010117, 100700845), 1), ((900124, 900160, 900171, 900233, 22010117, 100700845), 1), ((119, 900124, 900149, 900160, 900171, 900233), 1), ((119, 900124, 900149, 900160, 900171, 22010117), 1), ((119, 900124, 900149, 900160, 900171, 100700845), 1), ((114, 900101, 900236, 900237, 22010039, 100700841), 1), ((900142, 900152, 900212, 900222, 900244, 100700866), 1), ((631795, 801710, 900124, 900207, 900243, 22010083), 1)]
-----------------loop start----------------------


...

Itemsets of size  8 , count:  8032
sample: 
[((119, 900124, 900149, 900160, 900171, 900233, 22010117, 100700845), 1), ((144, 230103, 631795, 801710, 900124, 900218, 900243, 22010083), 1), ((144, 230103, 631795, 900102, 900124, 900218, 900243, 22010083), 1), ((631795, 801710, 900102, 900124, 900207, 900218, 900243, 22010083), 1), ((230103, 801710, 900102, 900124, 900207, 900218, 900243, 22010083), 1), ((144, 230103, 801710, 900124, 900207, 900218, 900243, 22010083), 1), ((144, 230103, 900102, 900124, 900207, 900218, 900243, 22010083), 1), ((144, 631795, 801710, 900124, 900207, 900218, 900243, 22010083), 1), ((144, 631795, 900102, 900124, 900207, 900218, 900243, 22010083), 1), ((144, 230103, 631795, 801710, 900102, 900124, 900207, 900243), 1)]
-----------------loop start----------------------


Itemsets of size  9 , count:  5806
sample: 
[((144, 230103, 631795, 801710, 900102, 900124, 900207, 900243, 22010083), 1), ((144, 230103, 631795, 801710, 900102, 900207, 900218, 900243, 22010083), 1), ((230103, 631795, 801710, 900102, 900124, 900207, 900218, 900243, 22010083), 1), ((144, 230103, 801710, 900102, 900124, 900218, 900244, 900277, 22010083), 1), ((144, 631795, 801710, 900102, 900124, 900218, 900244, 900277, 22010083), 1), ((144, 230103, 631795, 900124, 900207, 900218, 900244, 900277, 22010083), 1), ((230103, 631795, 801710, 900102, 900207, 900218, 900244, 900277, 22010083), 1), ((631795, 801710, 900124, 900207, 900218, 900243, 900244, 900277, 22010083), 1), ((631795, 900102, 900124, 900207, 900218, 900243, 900244, 900277, 22010083), 1), ((230103, 801710, 900124, 900207, 900218, 900243, 900244, 900277, 22010083), 1)]
-----------------loop start----------------------


Itemsets of size  10 , count:  3303
sample: 
[((144, 230103, 801710, 900102, 900124, 900207, 900218, 900244, 900277, 22010083), 1), ((144, 631795, 801710, 900102, 900124, 900207, 900218, 900244, 900277, 22010083), 1), ((144, 230103, 631795, 801710, 900207, 900218, 900243, 900244, 900277, 22010083), 1), ((144, 230103, 631795, 900102, 900207, 900218, 900243, 900244, 900277, 22010083), 1), ((230103, 631795, 801710, 900124, 900207, 900218, 900243, 900244, 900277, 22010083), 1), ((230103, 631795, 900102, 900124, 900207, 900218, 900243, 900244, 900277, 22010083), 1), ((144, 801710, 900102, 900124, 900207, 900218, 900243, 900244, 900277, 22010083), 1), ((144, 230103, 631795, 801710, 900124, 900207, 900243, 900244, 900277, 22010083), 1), ((144, 230103, 631795, 900102, 900124, 900207, 900243, 900244, 900277, 22010083), 1), ((144, 230103, 631795, 801710, 900124, 900207, 900218, 900243, 900277, 22010083), 1)]
-----------------loop start----------------------


Itemsets of size  11 , count:  1444
sample: 
[((144, 230103, 631795, 801710, 900102, 900124, 900218, 900243, 900244, 900277, 22010083), 1), ((144, 230103, 631795, 801710, 900102, 900124, 900207, 900243, 900244, 900277, 100701266), 1), ((144, 230103, 631795, 801710, 900102, 900207, 900218, 900243, 900244, 900277, 100701266), 1), ((230103, 631795, 801710, 900102, 900124, 900207, 900218, 900243, 900244, 900277, 100701266), 1), ((144, 230103, 631795, 801710, 900102, 900124, 900207, 900218, 900243, 900277, 100701266), 1), ((144, 230103, 631795, 801710, 900124, 900207, 900218, 900243, 900244, 22010083, 100701266), 1), ((144, 230103, 631795, 900102, 900124, 900207, 900218, 900243, 900244, 22010083, 100701266), 1), ((144, 230103, 631795, 801710, 900102, 900124, 900207, 900218, 900277, 22010083, 100701266), 1), ((144, 230103, 801710, 900102, 900124, 900207, 900218, 900243, 900277, 22010083, 100701266), 1), ((144, 631795, 801710, 900102, 900124, 900207, 900218, 900243, 900277, 22010083, 10070126

Itemsets of size  12 , count:  468
sample: 
[((144, 230103, 631795, 801710, 900102, 900124, 900207, 900218, 900243, 900244, 900277, 22010083), 1), ((144, 230103, 631795, 801710, 900124, 900207, 900218, 900243, 900244, 900277, 22010083, 100701266), 1), ((144, 230103, 631795, 900102, 900124, 900207, 900218, 900243, 900244, 900277, 22010083, 100701266), 1), ((230101, 631356, 631831, 900101, 900104, 900152, 900212, 900213, 900217, 900236, 900258, 100700835), 1), ((230101, 631356, 631831, 900101, 900104, 900152, 900212, 900213, 900217, 900244, 900258, 100700835), 1), ((230101, 631356, 631831, 900101, 900104, 900142, 900152, 900212, 900236, 900244, 900258, 100700835), 1), ((230101, 631356, 631831, 900104, 900142, 900152, 900212, 900213, 900236, 900244, 900258, 100700835), 1), ((230101, 631356, 631831, 900104, 900142, 900152, 900212, 900217, 900236, 900244, 900258, 100700835), 1), ((230101, 631831, 900101, 900104, 900152, 900212, 900213, 900217, 900236, 900244, 900258, 100700835), 1), ((23010

In [ ]:
# local
# sample 0.01 -> thresh = 0.001 -> freq items = 352 -> ~7m, ~9m
# sample 1 -> thresh 0.02 -> freq items = 37 -> 5m
# sample 1 -> thresh 0.01 -> freq items = 108 -> 39m -> 6 pairs, 0 3ples

## A-Priori as a function

<h3 dir="rtl">
 این کد، الگوریتم را به صورت یک تابع و یکجا درآوره است، تا در مرحله‌ی بعدی از آن در الگوریتم 
 SON
 استفاده کنیم.  
</h3> 


In [36]:
def apriori(baskets_rdd, MIN_COUNT, verbose=False):
  frequent_items_rdd = baskets_rdd.flatMap(lambda basket: [((item,),1) for item in basket]).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] > MIN_COUNT)

  def generate_combinations(old_combinations):
    """
    Input old_frequent_itemsets, and create new candidates from it
    """

    def generate_combinations_util(old_combination):
      """
      lambda function that maps an old combination to a number of new candidate combinations
      """
      old_combination = old_combination[0]
      old_combination_max_item = old_combination[-1]

      # Can do here numpy way
      bigger_items = remaining_items[remaining_items>old_combination_max_item]
      new_candidates = []
      for x in bigger_items:
        new_candidates.append( old_combination + (x,) )

      return new_candidates

    remaining_items = np.array(old_combinations.flatMap(lambda x: x[0]).distinct().sortBy(lambda x: x).collect())

    new_combinations = old_combinations.flatMap(generate_combinations_util)
    return new_combinations

  def get_new_frequents(candidates):
    _candidates = candidates.collect()
    def count_freq(basket):
      candidates_present = []
      for candidate in _candidates:
        if set(candidate) <= set(basket):
          candidates_present.append( (candidate,1) )
      
      return candidates_present

    frequent_itemsets_rdd = baskets_rdd.flatMap(count_freq).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>MIN_COUNT)
    
    return frequent_itemsets_rdd

  if verbose:
    print('MIN_COUNT is: ', MIN_COUNT)
  k = 1
  frequent_itemsets_rdd = frequent_items_rdd
  frequent_itemsets_rdds = []
  while frequent_itemsets_rdd.count() != 0:
    frequent_itemsets_rdds.append(frequent_itemsets_rdd)
    if verbose:
      print('-----------------loop start----------------------')
    k += 1
    candidates = generate_combinations(frequent_itemsets_rdd)
    frequent_itemsets_rdd = get_new_frequents(candidates)
    if verbose:
      print('Itemsets of size ', k, ', count: ', frequent_itemsets_rdd.count())
      print('sample: ')
      print(frequent_itemsets_rdd.take(10))

  return frequent_itemsets_rdds



<h3 dir="rtl">
کد زیر برای امتحان کردن کارایی تابع 
apriori
است.  
</h3> 


In [38]:
import numpy as np

SAMPLE = True
SAMPLE_SIZE = 0.01
THRESHOLD = 0.001


baskets_rdd = rdd_group.map(lambda x: x[1])

if SAMPLE:
  baskets_rdd = baskets_rdd.sample(True, SAMPLE_SIZE)
  baskets_rdd = baskets_rdd.coalesce(10)
  baskets_rdd.cache()

BASKETS_COUNT = baskets_rdd.count()
BASKETS_COUNT

MIN_COUNT = int(THRESHOLD * BASKETS_COUNT)


frequent_itemsets_rdds = apriori(baskets_rdd, MIN_COUNT, verbose=True)

MIN_COUNT is:  15
-----------------loop start----------------------


Itemsets of size  2 , count:  811
sample: 
[((101301, 900101), 30), ((900102, 100701100), 36), ((900182, 100701100), 33), ((145, 100700841), 20), ((900222, 900228), 28), ((900222, 100700868), 171), ((100700866, 100700868), 47), ((900234, 900276), 34), ((900235, 100700871), 41), ((209103, 900235), 18)]
-----------------loop start----------------------


Itemsets of size  3 , count:  184
sample: 
[((900212, 900244, 22009977), 23), ((631765, 900164, 900276), 17), ((631765, 900164, 100700820), 35), ((631765, 900276, 100700820), 21), ((900101, 900259, 100700841), 35), ((900155, 900222, 100700868), 50), ((205802, 900215, 900234), 19), ((142, 900215, 900234), 16), ((205802, 212802, 900233), 16), ((900215, 900234, 900256), 23)]
-----------------loop start----------------------


Itemsets of size  4 , count:  11
sample: 
[((22010087, 22010088, 22010094, 22010095), 28), ((900101, 900212, 900244, 100700839), 16), ((900193, 900212, 900244, 100700839), 16), ((900102, 900142, 900212, 900244), 18), ((900142, 900202, 900212, 900244), 16), ((900142, 900212, 900244, 900249), 17), ((900142, 900212, 900244, 100700853), 54), ((209103, 900265, 100700804, 100700834), 21), ((900142, 900152, 900212, 900244), 18), ((231, 900236, 900255, 100700841), 20)]
-----------------loop start----------------------
Itemsets of size  5 , count:  0
sample: 
[]


# SON

<h3 dir="rtl">
کد این قسمت، با استفاده از قسمت قبلی زده شده است. 

در مورد 
sample
و حداقل ساپورت توضیحات کاملا یکسان است. به هر کدام از ردیف‌های 
RDD
به صورت تصادفی عددی بین ۰ تا تعداد 
partiotion
ها منهی ۱ نسبت می‌دهم، که تعداد 
partiotion
ها نشان می‌دهد که می‌خواهم
RDD
خود را به چند قسمت تقسیم کنم. سپس، با فیلتر کردن روی عدد نسبت داده شده، 
RDD
من به تعداد
partiotion
قسمت تقریبا هم اندازه تقسیم می‌شود. حداقل ساپورت هر کدام از این قسمت‌ها را با توجه به اندازه‌شان (که تقریبا برابر هم است) به دست می‌آوریم، دقت کنید که چون اندازه‌ی این قسمت‌ها کوچک شده است،‌حداقل ساپورت نیز به نسبت کم شده است. برای اطمینان نیز حداقل ساپورت را در 
0.9
ضرب می‌کنیم تا چیزی را از دست ندهیم. حال الگوریتم
apriori
را روی تمامی قسمت‌ها اجرا می‌کنیم. سپس، خروجی این الگوریتم‌ها را با هم 
union
می‌کنیم، تا 
itemset
های پرتکرار برایند الگوریتم‌ها را به دست آوریم. اما این 
itemset
ها ممکن است حاوی
<span dir="ltr">
 false positive 
</span> 
باشند. برای همین یک بار دیگر روی کل دیتا حرکت می‌کنیم و این کاندید‌ها را دوباره می‌شماریم و فیلتر می‌کنیم. این قسمت دقیقا شبیه کار مشابه ما در الگوریتم
apriori
انجام شده است. نتایج به دست آمده از این الگوریتم، دقیقا با الگوریتم 
apriori
یکسان بود (که همین انتظار را نیز داشتیم، زیرا
SON
نه 
<span dir="ltr">
 false positive  
</span> 
دارد و نه 
<span dir="ltr">
 false negative 
</span> 
). بنابراین 
نتایج هردو قسمت را در یک بلاک یکسان در انتهای گزارش آورده‌ام. 

</h3> 


In [ ]:
import random

baskets_rdd = rdd_group.map(lambda x: x[1])

SAMPLE = False
SAMPLE_SIZE = 0.01
# THRESHOLD = 0.01
THRESHOLD = 0.02

if SAMPLE:
  baskets_rdd = baskets_rdd.sample(True, SAMPLE_SIZE)
  baskets_rdd = baskets_rdd.coalesce(10)
  baskets_rdd.cache()

BASKETS_COUNT = baskets_rdd.count()
BASKETS_COUNT

MIN_COUNT = int(THRESHOLD * baskets_rdd.count())
print('MIN_COUNT is: ', MIN_COUNT)
partitions_count = 4
def random_lambda(x):
    return (x, random.randint(0,partitions_count-1))

baskets_rdd_with_random = baskets_rdd.map(random_lambda)

# print(baskets_rdd.take(10))

baskets_rdds = []
for i in range(partitions_count):
  baskets_rdds.append(baskets_rdd_with_random.filter(lambda x: x[1] == i).map(lambda x: x[0]))


MIN_COUNTS = [ int(THRESHOLD * basket_rdd.count()) for basket_rdd in baskets_rdds]
print('MIN_COUNTS are: ', MIN_COUNTS)

frequent_itemsets_rdds_for_samples = []
for i in range(partitions_count):
  frequent_itemsets_rdds_for_samples.append(apriori(baskets_rdds[i], int(MIN_COUNTS[i]*0.9)))

max_size = max(list(map(len, frequent_itemsets_rdds_for_samples)))


for i in range(1, max_size+1):
  frequent_itemsets_rdds_for_samples = list(filter(lambda x: len(x) >= i, frequent_itemsets_rdds_for_samples))

  frequent_itemsets_rdd = sc.union([x[i-1] for x in frequent_itemsets_rdds_for_samples]).map(lambda x: x[0]).distinct()
  _candidates = frequent_itemsets_rdd.collect()
  def count_freq(basket):
    candidates_present = []
    for candidate in _candidates:
      if set(candidate) <= set(basket):
        candidates_present.append( (candidate,1) )
    
    return candidates_present

  frequent_itemsets_rdd = baskets_rdd.flatMap(count_freq).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>MIN_COUNT)
 
  
  print('Itemsets of size ', i, ', count: ', frequent_itemsets_rdd.count())
  print('sample: ')
  print(frequent_itemsets_rdd.take(10))


# Results

<h3 dir="rtl">
روی داده‌های کامل 
sample
داده شده،‌ با استفاده از
THRESHOLD
برابر با ۲ درصد، یا به عبارتی حداقل ساپورت برابر با 
۳۰۴۰۸،
item
های پرتکرار ۲۷ تا بودند، که یک نمونه ۱۰ تایی از آن‌ها 

 <br> 

<span dir="ltr">
((900107,), 38371), ((900155,), 45958), ((900207,), 35631), ((22010119,), 30749), ((900139,), 31477), ((900191,), 31379), ((100700824,), 34662), ((900244,), 61075), ((900164,), 37506), ((900236,), 41652)
</span> 

است. مجموعه‌ی پرتکرار ۲ تایی نیز تنها 
<span dir="ltr">
 ((900212, 900244), 42733)
</span> 
 وجود داشت (عدد سمت راست هر مجموعه تعداد تکرار آن است). 


 برای این که کدم را برای مجموعه‌های بزرگتر نیز تست کنم، از سمپل داده شده، یک سمپل 
 0.01
  نیز گرفتم، و 
 THRESHOLD
را نیز برابر 
 0.001
  قرار دادم. نتیجه‌ی این اجرا نیز به صورت زیر بود:  
</h3> 


<h4 >
  

SAMPLE_SIZE = 0.01

THRESHOLD = 0.001

MIN_COUNT is:  15



Itemsets of size  2 , count:  811

sample: 

[((101301, 900101), 30), ((900102, 100701100), 36), ((900182, 100701100), 33), ((145, 100700841), 20), ((900222, 900228), 28), ((900222, 100700868), 171), ((100700866, 100700868), 47), ((900234, 900276), 34), ((900235, 100700871), 41), ((209103, 900235), 18)]



Itemsets of size  3 , count:  184

sample: 
[((900212, 900244, 22009977), 23), ((631765, 900164, 900276), 17), ((631765, 900164, 100700820), 35), ((631765, 900276, 100700820), 21), ((900101, 900259, 100700841), 35), ((900155, 900222, 100700868), 50), ((205802, 900215, 900234), 19), ((142, 900215, 900234), 16), ((205802, 212802, 900233), 16), ((900215, 900234, 900256), 23)]



Itemsets of size  4 , count:  11

sample: 
[((22010087, 22010088, 22010094, 22010095), 28), ((900101, 900212, 900244, 100700839), 16), ((900193, 900212, 900244, 100700839), 16), ((900102, 900142, 900212, 900244), 18), ((900142, 900202, 900212, 900244), 16), ((900142, 900212, 900244, 900249), 17), ((900142, 900212, 900244, 100700853), 54), ((209103, 900265, 100700804, 100700834), 21), ((900142, 900152, 900212, 900244), 18), ((231, 900236, 900255, 100700841), 20)]

</h4> 